# Procesamiento de Datos Climatológicos - Estación Augusto Weberbauer (Cajamarca, Perú)

Este notebook procesa un archivo Excel con datos climatológicos registrados por el SENAMHI en la estación **Augusto Weberbauer**, ubicada en Cajamarca, Perú. El periodo de observación abarca desde **1994 hasta 2024**.

## Objetivo

Transformar y limpiar los datos meteorológicos registrados en formato mensual, para generar un dataset diario unificado, adecuado para análisis climáticos, modelado estadístico y aplicaciones de ciencia de datos.

## Variables procesadas

Se procesan las siguientes variables:
- Precipitación diaria
- Temperatura media diaria
- Humedad relativa
- Presión atmosférica
- Velocidad del viento
- Horas de sol

## Etapas del procesamiento

1. Lectura de hojas individuales desde el archivo Excel.
2. Reemplazo de valores:
   - `'T'` (trazas de precipitación) por `0.045`
   - `'S/D'` (sin dato) por `NaN`
3. Conversión de columnas mensuales al formato largo (*tidy*).
4. Unión de todas las variables en un solo DataFrame.
5. Exportación del conjunto limpio para análisis posteriores.

## Repositorio del proyecto

GitHub: [https://github.com/evalcas/climate-variables-senamhi-augusto-weberbauer-cajamarca-1994-2024](https://github.com/evalcas/climate-variables-senamhi-augusto-weberbauer-cajamarca-1994-2024)

## Autor y licencia

- Autor: **Edwin Valencia-Castillo**
- Fecha: **04 de julio de 2025**
- Licencia: [CC BY 4.0](https://creativecommons.org/licenses/by/4.0/)

## Cita

> Edwin Valencia-Castillo, & Sandra Rodriguez-Avila. (2025). Daily Climate Variables from the SENAMHI Augusto Weberbauer Station (Cajamarca, Peru), 1994–2024 (v1.0.0). Zenodo. https://doi.org/10.5281/zenodo.15803716


In [16]:
# Importa la biblioteca pandas para el manejo y análisis de estructuras de datos (DataFrames, Series, etc.)
import pandas as pd

# Importa la biblioteca numpy para realizar operaciones matemáticas y trabajar con arreglos y valores nulos (NaN)
import numpy as np

# Importa el módulo de advertencias para controlar la visualización de warnings durante la ejecución
import warnings

# Ignora específicamente las advertencias de tipo FutureWarning que podrían aparecer al utilizar funciones
# que cambiarán en versiones futuras de bibliotecas como pandas o numpy.
warnings.filterwarnings("ignore", category=FutureWarning)


In [17]:
# Función para procesar una hoja del archivo Excel climatológico
def procesar_hoja(df, nombre_columna):

    """
    Preprocesa una hoja de datos climatológicos del archivo Excel original.

    Operaciones realizadas:
    - Reemplaza los valores 'T' por 0.045, asumiendo una traza de precipitación (< 0.1 mm).
    - Reemplaza los valores 'S/D' (sin dato) por NaN.
    - Convierte todas las columnas de meses a valores numéricos (coerción de errores).
    - Reestructura el DataFrame en un formato largo (tidy), con las columnas: YEAR, MES, DIA, <nombre_columna>.

    Parámetros:
    - df (pd.DataFrame): DataFrame original leído desde una hoja del archivo Excel.
    - nombre_columna (str): Nombre final de la variable climatológica (ej., 'PrecipitacionDiaria').

    Retorna:
    - pd.DataFrame: Datos procesados y reestructurados.
    """

    df = df.replace('T', 0.045) # Reemplaza traza de precipitación
    df = df.replace('S/D', np.nan) # Reemplaza valores faltantes por NaN


    # Convierte valores mensuales a tipo numérico (manejo de errores)
    for col in df.columns[2:]:  # Desde la tercera columna (ENERO en adelante)
      df[col] = pd.to_numeric(df[col], errors='coerce')

    # Mapeo de nombres de meses a números
    columnas_meses = ['ENERO', 'FEBRERO', 'MARZO', 'ABRIL', 'MAYO', 'JUNIO', 'JULIO', 'AGOSTO', 'SETIEMBRE', 'OCTUBRE', 'NOVIEMBRE', 'DICIEMBRE']
    meses_num = list(range(1,13))
    meses_dict = dict(zip(columnas_meses, meses_num))

    # Reestructura los datos en formato largo
    df_final = pd.DataFrame()
    for mes, num_mes in meses_dict.items():
        temp_df = df[['YEAR', 'DIA', mes]].copy()
        temp_df['MES'] = num_mes
        temp_df = temp_df.rename(columns={mes: nombre_columna})
        df_final = pd.concat([df_final, temp_df], ignore_index=True)

    return df_final

# -----------------------------------
# Carga y procesamiento del archivo Excel
# -----------------------------------

# URL del archivo Excel en GitHub
url = 'https://github.com/evalcas/climate-variables-senamhi-augusto-weberbauer-cajamarca-1994-2024/blob/main/dataset_raw_senamhi30years.xlsx?raw=true'
excel_file = url

# Nombres de las hojas del archivo Excel (una por variable climática)
hojas = ['PRECIPITACION TOTAL DIARIA', 'TEMPERATURA MEDIA', 'HUMEDAD RELATIVA', 'PRESION ATMOSFERICA', 'VELOCIDAD DE VIENTO', 'HORAS DE SOL']

# Nombres normalizados para las columnas en el DataFrame final
nombres_columnas = ['PrecipitacionDiaria', 'TemperaturaMedia', 'HumedadRelativa', 'PresionAtmosferica', 'VelocidadViento', 'HorasSol']

# Diccionario donde se almacenarán los DataFrames por variable
dfs = {}

# Procesamiento individual de cada hoja del Excel
for hoja, nombre_columna in zip(hojas, nombres_columnas):
  try:
    df = pd.read_excel(excel_file, sheet_name=hoja)
    dfs[nombre_columna] = procesar_hoja(df, nombre_columna)
  except Exception as e:
    print(f"Error procesando la hoja '{hoja}': {e}")

# -----------------------------------
# Fusión de todas las variables en un solo DataFrame
# -----------------------------------

# Inicializa el DataFrame final con la primera variable
df_final = dfs[nombres_columnas[0]]

# Fusiona progresivamente las demás variables
for nombre_columna in nombres_columnas[1:]:
  if nombre_columna in dfs: # Verifica si existe
    df_final = pd.merge(df_final, dfs[nombre_columna], on=['YEAR', 'DIA', 'MES'], how='outer')

# Agrupar por año, mes y día
df_final = df_final.groupby(['YEAR', 'MES', 'DIA']).agg({col: 'mean' for col in nombres_columnas}).reset_index()

# Visualiza las primeras filas
print(df_final.head())



   YEAR  MES  DIA  PrecipitacionDiaria  TemperaturaMedia  HumedadRelativa  \
0  1994    1    1                  0.2              14.9             76.2   
1  1994    1    2                  7.7              15.1             77.8   
2  1994    1    3                  0.0              14.0             73.1   
3  1994    1    4                  0.0              15.3             66.9   
4  1994    1    5                  5.1              15.1             72.0   

   PresionAtmosferica  VelocidadViento  HorasSol  
0               740.1              1.3       6.8  
1               739.9              1.3       1.6  
2               739.5              1.7       6.8  
3               740.2              NaN       3.5  
4               740.1              2.0       6.2  


In [18]:
# Hacer una copia del DataFrame final para trabajar sobre ella sin modificar el original
df_completo = df_final.copy()

# Convertir las columnas 'YEAR', 'MES' y 'DIA' a tipo string para facilitar la concatenación posterior
df_completo['YEAR'] = df_completo['YEAR'].astype(str)
df_completo['MES'] = df_completo['MES'].astype(str)
df_completo['DIA'] = df_completo['DIA'].astype(str)

# Crear una nueva columna llamada 'ProcessDate' uniendo las columnas de año, mes y día con el formato 'YYYY-MM-DD'
df_completo['ProcessDate'] = df_completo['YEAR'] + '-' + df_completo['MES'] + '-' + df_completo['DIA']

# Convertir la columna 'ProcessDate' a formato datetime;
# si hay errores en el formato (por ejemplo, fechas inválidas como 2024-02-30), se marcarán como NaT (Not a Time)
# Luego se eliminan las filas con fechas inválidas
try:
    df_completo['ProcessDate'] = pd.to_datetime(df_completo['ProcessDate'], format='%Y-%m-%d', errors='coerce')
    df_completo = df_completo.dropna(subset=['ProcessDate'])
except Exception as e:
    print(f"Error al convertir a datetime: {e}")

# Renombrar las columnas 'YEAR', 'MES' y 'DIA' a 'Year', 'Month' y 'Day' para mejorar la legibilidad y consistencia
df_completo = df_completo.rename(columns={'YEAR': 'Year', 'MES': 'Month', 'DIA': 'Day'})


In [19]:
# Filtra las filas donde 'ProcessDate' contiene valores no convertidos (NaT)
# Esto puede suceder cuando la conversión a datetime falla y se asigna un valor especial 'NaT' (Not a Time)
# Se convierte la columna 'ProcessDate' a string para poder buscar el patrón 'NaT' en sus valores

filas_con_nat = df_completo[df_completo['ProcessDate'].astype(str).str.contains('NaT')]
filas_con_nat


,Year,Month,Day,PrecipitacionDiaria,TemperaturaMedia,HumedadRelativa,PresionAtmosferica,VelocidadViento,HorasSol,ProcessDate


In [21]:
# Renombra las columnas del DataFrame al inglés para mejorar la legibilidad y estandarizar
# los nombres en el contexto de una investigación científica internacional.
# Este paso facilita la comprensión por parte de audiencias no hispanohablantes
# y asegura consistencia con estándares internacionales en publicaciones.

df_completo = df_completo.rename(columns={
    'PrecipitacionDiaria': 'DailyPrecipitation',      # Precipitación total diaria
    'TemperaturaMedia': 'AverageTemperature',         # Temperatura media diaria
    'HumedadRelativa': 'RelativeHumidity',            # Humedad relativa
    'PresionAtmosferica': 'AtmosphericPressure',      # Presión atmosférica
    'VelocidadViento': 'WindSpeed',                   # Velocidad del viento
    'HorasSol': 'SunshineHours'                       # Horas de sol
})

# Muestra las primeras filas del DataFrame para verificar los nuevos nombres de columna
print(df_completo.head())

   Year Month Day  DailyPrecipitation  AverageTemperature  RelativeHumidity  \
0  1994     1   1                 0.2                14.9              76.2   
1  1994     1   2                 7.7                15.1              77.8   
2  1994     1   3                 0.0                14.0              73.1   
3  1994     1   4                 0.0                15.3              66.9   
4  1994     1   5                 5.1                15.1              72.0   

   AtmosphericPressure  WindSpeed  SunshineHours ProcessDate  
0                740.1        1.3            6.8  1994-01-01  
1                739.9        1.3            1.6  1994-01-02  
2                739.5        1.7            6.8  1994-01-03  
3                740.2        NaN            3.5  1994-01-04  
4                740.1        2.0            6.2  1994-01-05  


In [22]:
# Reorganiza las columnas del DataFrame colocando 'ProcessDate' como la primera columna.
# Esto se hace para facilitar el análisis temporal, ya que tener la fecha en la primera posición
# permite una visualización y manipulación más intuitiva del DataFrame.
cols = list(df_completo.columns)                            # Obtiene la lista de nombres de columnas
cols.insert(0, cols.pop(cols.index('ProcessDate')))         # Mueve 'ProcessDate' al inicio de la lista
df_completo = df_completo.loc[:, cols]                      # Reordena las columnas del DataFrame

# Muestra el contenido actualizado del DataFrame
df_completo


,ProcessDate,Year,Month,Day,DailyPrecipitation,AverageTemperature,RelativeHumidity,AtmosphericPressure,WindSpeed,SunshineHours
0,1994-01-01,1994,1,1,0.2,14.9,76.2,740.1,1.3,6.8
1,1994-01-02,1994,1,2,7.7,15.1,77.8,739.9,1.3,1.6
2,1994-01-03,1994,1,3,0.0,14.0,73.1,739.5,1.7,6.8
3,1994-01-04,1994,1,4,0.0,15.3,66.9,740.2,NaN,3.5
4,1994-01-05,1994,1,5,5.1,15.1,72.0,740.1,2.0,6.2
...,...,...,...,...,...,...,...,...,...,...
11527,2024-12-27,2024,12,27,11.3,14.0,88.6,740.2,0.3,1.5
11528,2024-12-28,2024,12,28,9.9,15.2,79.8,740.7,0.3,2.3
11529,2024-12-29,2024,12,29,6.6,16.4,71.0,740.4,0.7,1.7
11530,2024-12-30,2024,12,30,0.2,15.8,71.5,739.5,1.3,7.4


In [23]:
# --- Verifica la completitud temporal del DataFrame ---
# Obtiene la fecha más antigua y más reciente del DataFrame en la columna 'ProcessDate'.
fechas_minima = df_completo['ProcessDate'].min()
fechas_maxima = df_completo['ProcessDate'].max()

# Genera un rango completo de fechas diarias entre la fecha mínima y máxima.
rango_fechas = pd.date_range(start=fechas_minima, end=fechas_maxima)

# Identifica las fechas que están en el rango completo pero no en el DataFrame original.
fechas_faltantes = rango_fechas.difference(df_completo['ProcessDate'])


# Muestra las fechas faltantes, si las hubiera.
print("Fechas faltantes:")
print(fechas_faltantes)

# Muestra el resumen del rango de fechas del DataFrame.
print("\nFecha más antigua:", fechas_minima)
print("Fecha más reciente:", fechas_maxima)


Fechas faltantes:
DatetimeIndex([], dtype='datetime64[ns]', freq='D')

Fecha más antigua: 1994-01-01 00:00:00
Fecha más reciente: 2024-12-31 00:00:00


In [24]:
# --- Identificación de registros completamente vacíos en las variables climáticas ---

# Filtra las filas en las que **todas** las columnas de variables climáticas son NaN (valores perdidos),
# excluyendo las columnas 'ProcessDate', 'Year', 'Month' y 'Day' que sirven como identificadores temporales.
# Este análisis permite detectar fechas para las cuales no se registró **ninguna** variable, lo cual podría
# deberse a eventos como la pandemia u otros problemas en la recolección de datos.
filas_todos_nan = df_completo[df_completo.drop(columns=['ProcessDate', 'Year', 'Month', 'Day']).isna().all(axis=1)]

# Muestra dichas filas para su posterior revisión o tratamiento mediante técnicas de imputación.
filas_todos_nan


,ProcessDate,Year,Month,Day,DailyPrecipitation,AverageTemperature,RelativeHumidity,AtmosphericPressure,WindSpeed,SunshineHours
5549,2008-12-01,2008,12,1,NaN,NaN,NaN,NaN,NaN,NaN
5550,2008-12-02,2008,12,2,NaN,NaN,NaN,NaN,NaN,NaN
5551,2008-12-03,2008,12,3,NaN,NaN,NaN,NaN,NaN,NaN
5552,2008-12-04,2008,12,4,NaN,NaN,NaN,NaN,NaN,NaN
5553,2008-12-05,2008,12,5,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
9852,2020-06-26,2020,6,26,NaN,NaN,NaN,NaN,NaN,NaN
9853,2020-06-27,2020,6,27,NaN,NaN,NaN,NaN,NaN,NaN
9854,2020-06-28,2020,6,28,NaN,NaN,NaN,NaN,NaN,NaN
9855,2020-06-29,2020,6,29,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
from google.colab import files

# Guarda el DataFrame como un archivo Excel en el entorno temporal de Colab
df_completo.to_excel('dataset_preprocessed_senamhi30Year.xlsx', index=False)

# Descarga el archivo a la computadora local del usuario
files.download('dataset_preprocessed_senamhi30Year.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>